### 1) 라이브러리 설치

In [1]:
!poetry add pypdf=">=4.2.0,<5.0.0"
!poetry add langchain-upstage


Updating dependencies
Resolving dependencies...

No dependencies to install or update

Writing lock file
The following packages are already present in the pyproject.toml and will be skipped:

  - langchain-upstage

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.


#### PyPDFLoader 간단한 예제

In [4]:
from dotenv import load_dotenv
import os

# .env 파일을 불러와서 환경 변수로 설정
load_dotenv()

UPSTAGE_API_KEY = os.getenv("UPSTAGE_API_KEY")
print(UPSTAGE_API_KEY[30:])

w2


#### Upstage 를 사용한 RAG
* ChatUpstage
* UpstageEmbeddings
* UpstageDocumentParseLoader

In [5]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_upstage import ChatUpstage
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_upstage import UpstageEmbeddings

print("==> 1. 문서 로딩 → PDF 읽기...")
file_path = "../data/tutorial-korean.pdf"
loader = PyPDFLoader('../data/tutorial-korean.pdf')
documents = loader.load()
print(f"  총 {len(documents)}페이지 로드 완료")


==> 1. 문서 로딩 → PDF 읽기...
  총 39페이지 로드 완료


In [8]:

print("==> 2. 문서 분할 → 작은 청크로 나누기")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 청크 크기 (한국어 최적화)
    chunk_overlap=200,      # 중복 부분 (맥락 보존)
    separators=["\n\n", "\n", ".", " ", ""] # 자연스러운 분할을 위한 구분자
)
chunks = text_splitter.split_documents(documents)
print(f"  {len(chunks)}개 청크 생성 완료")
print(f"  평균 청크 길이: {sum(len(chunk.page_content) for chunk in chunks) / len(chunks):.0f}자")

print(type(chunks),chunks)


==> 2. 문서 분할 → 작은 청크로 나누기
  66개 청크 생성 완료
  평균 청크 길이: 688자
<class 'list'> [Document(metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7D1B1DBC6A9C5E4B8AEBEF3B9AEBCAD283230292E687770>', 'source': '../data/tutorial-korean.pdf', 'total_pages': 39, 'page': 0, 'page_label': '1'}, page_content='1BlueJ 튜토리얼                   \n한국어 버전 2.0\nBlueJ Version 2.0.x 用\n                       \n          \nEnglish Version 2.0.1\nMichael Kölling\nMærsk Insitute\nUniversity of Southern Denmark \nKorean Version 2.0\n황석형 교수\nProject Member\nVer. 1.0 : 강석진, 민상호, 오경묵, 유형순, 이승환. Ver. 2.0 : 강원준\n 선문대학교 컴퓨터정보학부'), Document(metadata={'producer': 'Acrobat Distiller with ezUniHFT', 'creator': 'PScript5.dll Version 5.2', 'creationdate': '2005-04-26T15:21:34+09:00', 'moddate': '2005-04-26T15:21:34+09:00', 'author': 'Owner', 'title': '<426C75654AC7

In [9]:


print("==> 3. 벡터화 → 임베딩으로 변환")
embeddings = UpstageEmbeddings(model="solar-embedding-1-large")

print("==> 4. 저장 → FAISS 벡터스토어에 저장")
vectorstore = FAISS.from_documents(chunks, embeddings)
print(f" FAISS 벡터스토어 생성 완료 ({len(chunks)}개 벡터)")
# 로컬 파일로 저장
vectorstore.save_local("faiss_index_upstage")

print("===> 5. 검색 → 질문과 유사한 문서 찾기")
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 6}  # 상위 6개 관련 문서 검색
)
print(" Retriever 설정 완료")

==> 3. 벡터화 → 임베딩으로 변환
==> 4. 저장 → FAISS 벡터스토어에 저장
 FAISS 벡터스토어 생성 완료 (66개 벡터)
===> 5. 검색 → 질문과 유사한 문서 찾기
 Retriever 설정 완료


In [10]:

print("===> 6. 생성 → LLM으로 답변 생성")
llm = ChatUpstage(
        model="solar-pro",
        base_url="https://api.upstage.ai/v1",
        temperature=0.5
    )
print(llm)

===> 6. 생성 → LLM으로 답변 생성
client=<openai.resources.chat.completions.completions.Completions object at 0x000001992F2D5A90> async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x000001992F2D5D30> model_name='solar-pro' temperature=0.5 model_kwargs={} upstage_api_key=SecretStr('**********') upstage_api_base='https://api.upstage.ai/v1'


In [11]:

# 한국어 최적화 프롬프트
prompt_template = """
당신은 BlueJ 프로그래밍 환경 전문가입니다. 
아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 규칙:
1. 문서 내용만을 근거로 답변하세요
2. 단계별 설명이 필요하면 순서대로 작성하세요  
3. 구체적인 메뉴명, 버튼명을 포함하세요
4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요

답변:"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["context", "question"]
)
print(" 프롬프트 설정 완료")
print(prompt)


 프롬프트 설정 완료
input_variables=['context', 'question'] input_types={} partial_variables={} template='\n당신은 BlueJ 프로그래밍 환경 전문가입니다. \n아래 문서 내용을 바탕으로 정확하고 친절한 답변을 제공해주세요.\n\n문서 내용:\n{context}\n\n질문: {question}\n\n답변 규칙:\n1. 문서 내용만을 근거로 답변하세요\n2. 단계별 설명이 필요하면 순서대로 작성하세요  \n3. 구체적인 메뉴명, 버튼명을 포함하세요\n4. 문서에 없는 정보는 "문서에서 찾을 수 없습니다"라고 하세요\n\n답변:'


In [12]:

# ===================================
# 7. QA 체인 생성
# ===================================
print("\n ===> 7.  QA 체인 생성...")
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)
print("  RAG 파이프라인 구축 완료!")

# ===================================
# 8. 테스트 질문들
# ===================================
test_questions = [
    "BlueJ에서 객체를 생성하는 방법은 무엇인가요?",
    "컴파일 오류가 발생했을 때 어떻게 확인할 수 있나요?", 
    "디버깅을 위해 중단점을 설정하는 방법을 알려주세요",
    "코드패드는 무엇이고 어떻게 사용하나요?",
    "애플릿을 만들고 실행하는 방법을 설명해주세요"
]

print("\n" + "=" * 60)
print(" RAG 시스템 테스트")
print("=" * 60)

# ===================================
# 9. 질문 및 답변 실행
# ===================================
for i, question in enumerate(test_questions, 1):
    print(f"\n【테스트 {i}/5】")
    print(f" 질문: {question}")
    print(" 답변 생성 중...")
    
    # RAG 실행
    result = qa_chain.invoke({"query": question})
    answer = result["result"]
    source_docs = result["source_documents"]
    
    print(f"\n 답변:")
    print("-" * 50)
    print(answer)
    
    # 참조 문서 정보
    print(f"\n 참조 문서:")
    for j, doc in enumerate(source_docs[:3], 1):
        page = doc.metadata.get('page', 'N/A')
        preview = doc.page_content[:80].replace('\n', ' ')
        print(f"   {j}. 페이지 {page}: {preview}...")
    
    print("\n" + "-" * 40)


 ===> 7.  QA 체인 생성...
  RAG 파이프라인 구축 완료!

 RAG 시스템 테스트

【테스트 1/5】
 질문: BlueJ에서 객체를 생성하는 방법은 무엇인가요?
 답변 생성 중...

 답변:
--------------------------------------------------
BlueJ에서 객체를 생성하는 방법은 다음과 같습니다. 문서 내용을 바탕으로 단계별로 설명드리겠습니다.

---

### 1. **클래스 아이콘을 통한 객체 생성 (기본 방법)**
   - **단계 1**: 메인 윈도우 중앙의 클래스 아이콘(예: `Person`, `Staff`)에서 **마우스 오른쪽 버튼 클릭** (Mac의 경우 `Ctrl + Click`)합니다.  
   - **단계 2**: 팝업 메뉴에서 **생성자 함수**를 선택합니다.  
     - 대화상자가 나타나며, 생성할 객체의 이름을 입력받습니다. 기본 이름(예: `staff_1`)이 제공되므로 **OK 버튼**을 클릭해 객체를 생성합니다.  
   - **단계 3**: 생성된 객체는 **오브젝트 벤치(Object Bench)**에 표시됩니다(그림 4 참조).

> 📌 **참고**: 추상 클래스(`<<abstract>>` 표기)는 객체를 생성할 수 없습니다.

---

### 2. **라이브러리 클래스로부터 객체 생성**
   - **단계 1**: 상단 메뉴에서 **Tools → Use Libraries Class**를 선택합니다.  
   - **단계 2**: 팝업된 대화상자에 **패키지 이름까지 포함한 완전한 클래스명**을 입력합니다(예: `java.lang.String`, `java.util.ArrayList`).  
     - 최근 사용된 클래스 목록은 텍스트 필드에 자동 표시됩니다.  
   - **단계 3**: 엔터 키를 눌러 클래스의 **생성자 및 정적 메소드 목록**을 확인합니다.  
   - **단계 4**: 원하는 생성자 또는 정적 메소드를 선택해 호출합니다.

---

###

In [13]:
from langchain_upstage import UpstageGroundednessCheck

groundedness_check = UpstageGroundednessCheck()

question = "BlueJ에서 객체를 생성하는 방법은 무엇인가요?"

retrieved_docs = retriever.invoke(question)
context = "\n\n".join([doc.page_content for doc in retrieved_docs])
#print(context)

response = llm.invoke(f"{question} 관련 정보: {context}")

request_input = {
    "context": context,
    "answer": response.content
}

response = groundedness_check.invoke(request_input)
print(response)  

grounded
